# Initialization 

In [0]:
import pyspark.sql.functions as F
from pyspark.sql.types import StringType, DateType
from pyspark.sql.functions import trim, col

# Read Bronze Table

In [0]:
df = spark.table("workspace.bronze.erp_loc_a101")